In [ ]:
#%pip install -U "qwen-agent[gui,rag,code_interpreter,python_executor]"

### This is an implementation of the Qwen Agent framework using HF Inference Endpoints
For personal use only. I don't take any liabilities.

In [1]:
import pprint
import urllib.parse
import json5
from typing import Optional
from qwen_agent.agents import Assistant, ReActChat
from qwen_agent.tools.base import BaseTool, register_tool
from huggingface_hub import login 

#### In order to use the HF endpoint you need to be logged-in and properly authenticated.
Let me know if you need any help here.

In [2]:
hf_token = "<YOUR TOKEN>"
login(hf_token,add_to_git_credential=False) 

In [3]:
repo_id="Qwen/QwQ-32B-Preview"

In [4]:
files = ['./fraunhofer-emi-annual-report-2023-2024.pdf']  # Give the bot a PDF file to read.

In [5]:
# Step 1 (Optional): Add a custom tool named `my_image_gen`.
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # The `description` tells the agent the functionality of this tool.
    description = 'AI painting (image generation) service, input text description, and return the image URL drawn based on text information.'
    # The `parameters` tell the agent what input parameters the tool has.
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': 'Detailed description of the desired image content, in English',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` are the arguments generated by the LLM agent.
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)

In [6]:
# Step 2: Configure the LLM you are using.
# LLM Configuration using the HfApiEngine

llm_cfg = {
    'model': repo_id,
    'model_server': f'https://api-inference.huggingface.co/models/{repo_id}/v1',  # base_url, also known as api_base
    'api_key': hf_token,
    'generate_cfg': {
        'top_p': 0.8
    }
}


### The default system prompt -- but its not that good

In [7]:
# Step 3: Create an agent. Here we use the `Assistant` agent as an example, which is capable of using tools and reading files.
system_instruction = '''You are a helpful assistant.
After receiving the user's request, you should:
- first draw an image and obtain the image url,
- then run code `request.get(image_url)` to download the image,
- and finally select an image operation from the given document to process the image.
Please show the image using `plt.show()`.'''


In [8]:
#define tools and agent
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` is a built-in tool for executing code.

agent = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools ,files=files)



### The default example

In [ ]:
# Step 4: Run the agent as a chatbot.
messages = []  # This stores the chat history.
while True:
    # For example, enter the query "draw a dog and rotate it 90 degrees".
    query = input('user query: ')
    # Append the user query to the chat history.
    messages.append({'role': 'user', 'content': query})
    response = []
    for response in agent.run(messages=messages):
        # Streaming output.
        print('bot response:')
        pprint.pprint(response, indent=2)
    # Append the bot responses to the chat history.
    messages.extend(response)

user query: draw a dog and rotate it 90 degrees


2024-11-30 21:12:27,437 - split_query.py - 82 - INFO - Extracted info from query: {"information": ["draw a dog and rotate it 90 degrees
2024-11-30 21:12:30,680 - memory.py - 113 - INFO - {"keywords_zh": ["画一只狗", "旋转", "90度"], "keywords_en": ["draw a dog", "rotate", "90 degrees"], "text": "draw a dog and rotate it 90 degrees"}
2024-11-30 21:12:32,987 - doc_parser.py - 99 - INFO - Read chunked ./fraunhofer-emi-annual-report-2023-2024.pdf from cache.
2024-11-30 21:12:32,988 - base_search.py - 56 - INFO - all tokens: 68412


bot response:
[{'content': 'To', 'role': 'assistant'}]
bot response:
[{'content': 'To complete', 'role': 'assistant'}]
bot response:
[{'content': 'To complete this', 'role': 'assistant'}]
bot response:
[{'content': 'To complete this task', 'role': 'assistant'}]
bot response:
[{'content': 'To complete this task,', 'role': 'assistant'}]
bot response:
[{'content': 'To complete this task, I', 'role': 'assistant'}]
bot response:
[{'content': "To complete this task, I'll", 'role': 'assistant'}]
bot response:
[{'content': "To complete this task, I'll need", 'role': 'assistant'}]
bot response:
[{'content': "To complete this task, I'll need to", 'role': 'assistant'}]
bot response:
[{'content': "To complete this task, I'll need to follow", 'role': 'assistant'}]
bot response:
[ { 'content': "To complete this task, I'll need to follow a",
    'role': 'assistant'}]
bot response:
[ { 'content': "To complete this task, I'll need to follow a few",
    'role': 'assistant'}]
bot response:
[ { 'content':

bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               'the image and rotate',
    'role': 'assistant'}]
bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               'the image and rotate it',
    'role': 'assistant'}]
bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               'the image and rotate it by',
    'role': 'assistant'}]
bot response:
[ { 'content': "To complete this

bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               "the image and rotate it by 90 degrees. Finally, I'll display "
               'the rotated image.\n'
               '\n'
               '### Step 1: Generate the Image',
    'role': 'assistant'}]
bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               "the image and rotate it by 90 degrees. Finally, I'll display "
               'the rotated image.\n'
               '\n'
               '### Step 1: Generate the Image\n'
               '\n'
               'I',
    'role': 'assistant'}]
bot response:
[ { 'content': "To co

bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               "the image and rotate it by 90 degrees. Finally, I'll display "
               'the rotated image.\n'
               '\n'
               '### Step 1: Generate the Image\n'
               '\n'
               "I'll use the `my_image_gen` tool to create an image of a dog. "
               'I need',
    'role': 'assistant'}]
bot response:
[ { 'content': "To complete this task, I'll need to follow a few steps. First, "
               "I'll use the `my_image_gen` tool to generate an image of a "
               "dog. Then, I'll use the `code_interpreter` tool to download "
               "the image and rotate it by 90 degrees. Finally, I'll display "
               'the rotated image.\n'
               '\n'
               '### Step 

### The ReAct Agent example

In [10]:
tools = ['code_interpreter']  # `code_interpreter` is a built-in tool for executing code.

agent = ReActChat(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools ,files=files)


In [ ]:
messages = []  # This stores the chat history.
while True:
    # For example, enter the query "draw a dog and rotate it 90 degrees".
    query = input('user query: ')
    # Append the user query to the chat history.
    messages.append({'role': 'user', 'content': query})
    response = []
    
    response = list(agent.run(messages=messages))
    pprint.pprint(response[-1], indent=2)
    # Streaming output.
    
    # Append the bot responses to the chat history.
    messages.extend(response)

user query: What is the weather in Tokyo now?
[ { 'content': 'Thought: I need to find out the current weather in Tokyo. I '
               'think I can use an API to get this information. There are many '
               'weather APIs available, like OpenWeatherMap or Weatherstack. I '
               'should choose one that provides accurate and up-to-date '
               'information.\n'
               '\n'
               'First, I need to decide which API to use. OpenWeatherMap is '
               'popular and has a lot of features, but it might require '
               'signing up and getting an API key. Weatherstack also looks '
               'good, and they offer a free tier with a',
    'role': 'assistant'}]
